<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/Turkish/biGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Turkish Language

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Turkish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Turkish


In [3]:
ls

offenseval-annotation.txt      readme-trainingset-tr.txt
offenseval-tr-training-v1.tsv


In [0]:
import pandas as pd
train=pd.read_csv( 'offenseval-tr-training-v1.tsv',sep="\t")
train=train.sample(frac=1)

In [5]:
train.head()

,id,tweet,subtask_a
7918,12142,Babasızlık zor ben iyi bilirim.,NOT
30677,33896,@USER Burada “rakiplerini egale etme” ne anlam...,NOT
2980,10244,@USER Milletimizin başı sağolsun şehidimizin m...,NOT
14822,41201,bu saatte malatya 32 dereceymiş ağlıycam gerçe...,NOT
22660,46204,@USER kötü günler mi yaşıyorsun? Sakın bunlar...,NOT


In [6]:
train.tail()

,id,tweet,subtask_a
23676,44750,evde kış havası yaratmak için 18 derecede klim...,NOT
787,24210,Çantadan halloween böceği çıktı. Kısmetim açıl...,NOT
3946,15906,"Güven sarsılınca, saygı, değer de sarsılıyormu...",NOT
8811,33329,He öküz ve genç tayfası.Senin müthis tespitler...,OFF
12551,21226,Türkiye’nin ilk finansal blockchain projesi Bo...,NOT


In [0]:
labels=train['subtask_a']
train=train['tweet']
labels=pd.factorize(labels)[0]

In [8]:
import collections
collections.Counter(labels)

Counter({0: 25231, 1: 6046})

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Using TensorFlow backend.


In [10]:
len(sequences_matrix),len(labels)

(31277, 31277)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU,GRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
model = Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=max_len))
model.add(Bidirectional(CuDNNGRU(150, return_sequences = True)))
#model.add(Bidirectional(CuDNNGRU(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(50, activation="relu"))
model.add(Dense(20, activation="relu"))

model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=[f1])

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(sequences_matrix,labels,test_size=0.33,stratify=labels,shuffle=True)

In [0]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1)

In [38]:

model.fit(x_train,y_train,batch_size=256,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val),callbacks=[reduce_lr])

Train on 20955 samples, validate on 10322 samples
Epoch 1/10
 - 8s - loss: 0.6422 - f1: 0.3990 - val_loss: 0.5583 - val_f1: 0.5118
Epoch 2/10
 - 8s - loss: 0.5597 - f1: 0.5315 - val_loss: 0.4736 - val_f1: 0.5261
Epoch 3/10
 - 8s - loss: 0.4016 - f1: 0.6588 - val_loss: 0.5066 - val_f1: 0.5187
Epoch 4/10
 - 8s - loss: 0.3149 - f1: 0.7322 - val_loss: 0.5187 - val_f1: 0.5146

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0004000000189989805.
Epoch 5/10
 - 8s - loss: 0.2361 - f1: 0.8096 - val_loss: 0.5153 - val_f1: 0.5223
Epoch 6/10
 - 8s - loss: 0.2174 - f1: 0.8233 - val_loss: 0.5643 - val_f1: 0.5157

Epoch 00006: ReduceLROnPlateau reducing learning rate to 8.000000379979611e-05.
Epoch 7/10
 - 8s - loss: 0.1997 - f1: 0.8361 - val_loss: 0.5365 - val_f1: 0.5252
Epoch 8/10
 - 8s - loss: 0.1956 - f1: 0.8430 - val_loss: 0.5469 - val_f1: 0.5248

Epoch 00008: ReduceLROnPlateau reducing learning rate to 1.6000001050997525e-05.
Epoch 9/10
 - 8s - loss: 0.1923 - f1: 0.8483 - val_loss: 0

In [39]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_val, batch_size=128, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

10322/10322 [==============================] - 1s 140us/step
              precision    recall  f1-score   support

           0       0.89      0.86      0.87      8327
           1       0.49      0.57      0.53      1995

    accuracy                           0.80     10322
   macro avg       0.69      0.72      0.70     10322
weighted avg       0.82      0.80      0.81     10322



In [0]:
import keras
keras.backend.clear_session()